In [2]:
import numpy as np
import pandas as pd
import os
import dotenv
import psycopg
from sqlalchemy import create_engine

import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

import plotly.express as px
import plotly.figure_factory as ff

In [3]:
dotenv.load_dotenv()
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')

dbms = 'postgresql'
package = 'psycopg'
user = 'postgres'
password = POSTGRES_PASSWORD
host = 'localhost'
port = '5433'
db = 'congress'

engine = create_engine(f'{dbms}+{package}://{user}:{password}@{host}:{port}/{db}')

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']


In [4]:
my_query = '''
SELECT *
FROM members
'''

data = pd.read_sql(my_query, con=engine)

In [5]:
data_df = data[['full_name', 'state_abbrev', 'district_code', 'party']]
data_df['party'] = data_df['party'].str[0]
display_name = [n + ' (' + s + '-' + str(d) +')' for n, s, d in zip(data_df['full_name'], data_df['state_abbrev'], data_df['district_code'])]
display_name = [x.replace('-0', '') for x in display_name]
display_name


['Robert B. Aderholt (AL-4)',
 'Jake Auchincloss (MA-4)',
 'Mark E. Amodei (NV-2)',
 'Alma S. Adams (NC-12)',
 'Pete Aguilar (CA-33)',
 'Rick W. Allen (GA-12)',
 'Jodey C. Arrington (TX-19)',
 'Mark Alford (MO-4)',
 'Gabe Amo (RI-1)',
 'Yassamin Ansari (AZ-3)',
 'Angela D. Alsobrooks (MD)',
 'Sanford D. Bishop, Jr. (GA-2)',
 'Cliff Bentz (OR-2)',
 'Stephanie I. Bice (OK-5)',
 'Lauren Boebert (CO-4)',
 'Tammy Baldwin (WI)',
 'John Boozman (AR)',
 'Marsha Blackburn (TN)',
 'Gus M. Bilirakis (FL-12)',
 'Vern Buchanan (FL-16)',
 'John Barrasso (WY)',
 'Michael F. Bennet (CO)',
 'Richard Blumenthal (CT)',
 'Suzanne Bonamici (OR-1)',
 'Joyce Beatty (OH-3)',
 'Andy Barr (KY-6)',
 'Julia Brownley (CA-26)',
 'Ami Bera (CA-6)',
 'Cory A. Booker (NJ)',
 'Brian Babin (TX-36)',
 'Donald S. Beyer, Jr. (VA-8)',
 'Mike Bost (IL-12)',
 'Brendan F. Boyle (PA-2)',
 'Don Bacon (NE-2)',
 'Jim Banks (IN)',
 'Nanette Diaz Barragán (CA-44)',
 'Jack Bergman (MI-1)',
 'Andy Biggs (AZ-5)',
 'Lisa Blunt Rochester

In [6]:
data_df.head()

,full_name,state_abbrev,district_code,party
0,Robert B. Aderholt,AL,4,R
1,Jake Auchincloss,MA,4,D
2,Mark E. Amodei,NV,2,R
3,Alma S. Adams,NC,12,D
4,Pete Aguilar,CA,33,D


In [7]:
dropdown_options = [{'label': y, 'value': x} for x, y in zip(data['bioguide_id'], display_name)]
dropdown_options

[{'label': 'Robert B. Aderholt (AL-4)', 'value': 'A000055'},
 {'label': 'Jake Auchincloss (MA-4)', 'value': 'A000148'},
 {'label': 'Mark E. Amodei (NV-2)', 'value': 'A000369'},
 {'label': 'Alma S. Adams (NC-12)', 'value': 'A000370'},
 {'label': 'Pete Aguilar (CA-33)', 'value': 'A000371'},
 {'label': 'Rick W. Allen (GA-12)', 'value': 'A000372'},
 {'label': 'Jodey C. Arrington (TX-19)', 'value': 'A000375'},
 {'label': 'Mark Alford (MO-4)', 'value': 'A000379'},
 {'label': 'Gabe Amo (RI-1)', 'value': 'A000380'},
 {'label': 'Yassamin Ansari (AZ-3)', 'value': 'A000381'},
 {'label': 'Angela D. Alsobrooks (MD)', 'value': 'A000382'},
 {'label': 'Sanford D. Bishop, Jr. (GA-2)', 'value': 'B000490'},
 {'label': 'Cliff Bentz (OR-2)', 'value': 'B000668'},
 {'label': 'Stephanie I. Bice (OK-5)', 'value': 'B000740'},
 {'label': 'Lauren Boebert (CO-4)', 'value': 'B000825'},
 {'label': 'Tammy Baldwin (WI)', 'value': 'B001230'},
 {'label': 'John Boozman (AR)', 'value': 'B001236'},
 {'label': 'Marsha Black

In [8]:
b = 'A000055'

my_query = f'''
SELECT *
FROM members
WHERE bioguide_id = '{b}'
'''

member_info = pd.read_sql(my_query, con=engine)
member_info = member_info.drop(['bioguide_id', 'image', 'fec_id', 'bioname', 'icpsr'], axis = 1)
ff.create_table(member_info.T.reset_index().rename({'index': '', 0: ''}, axis = 1))

In [9]:
my_query = f'''
SELECT image
FROM members
WHERE bioguide_id = '{b}'
'''
pd.read_sql(my_query, con=engine)['image'][0]

'https://www.congress.gov/img/member/a000055_200.jpg'

In [13]:
my_query = f'''
SELECT 
    c.comparison_member,
    c.agree,
    m.left_right_ideology,
    m.party
FROM members m
INNER JOIN (
    SELECT 
        vc.comparison_member, 
        vc.agree
    FROM members m
    INNER JOIN vote_compare vc
        ON m.bioname = vc.bioname
    WHERE m.bioguide_id = '{b}') c
    ON m.bioname = c.comparison_member
'''

votes =pd.read_sql(my_query, con=engine)

In [16]:
import plotly.express as px

fig = px.scatter(votes, x = 'left_right_ideology', y = 'agree', color = 'party', color_discrete_map={'Democrat': 'blue', 'Republican': 'red'}, hover_name = 'comparison_member')
fig.show()